In [1]:
# 1 Layer DNN for MNIST
# Import dependencies
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
n_input = 784
n_classes = 10

validation_size = 2000

In [4]:
labels = train['label']
features = train.drop(['label'], axis = 1)

In [5]:
#One hot encode
labels = pd.get_dummies(labels)

In [6]:
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [7]:
valid_features = features[:validation_size]
valid_labels = labels[:validation_size]

train_features = features[validation_size:]
train_labels = labels[validation_size:]

In [8]:
test_features = (test.values).astype('float32')
print(test_features.shape)


(28000, 784)


In [9]:
print(train_features.shape)
print(train_labels.shape)
print(valid_features.shape)
print(valid_labels.shape)

(40000, 784)
(40000, 10)
(2000, 784)
(2000, 10)


In [10]:
#Make TF graph
num_units = 2048


In [11]:
#Define Layers and Model
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights & bias
w1 = tf.Variable(tf.random_normal([n_input, num_units]))
b1 = tf.Variable(tf.random_normal([num_units]))

# Hidden Layer - a(xW1 + b1)
z1 = tf.add(tf.matmul(features, w1), b1)
h1 = tf.nn.relu(z1)

# Weights & bias
w0 = tf.Variable(tf.random_normal([num_units, n_classes]))
b0 = tf.Variable(tf.random_normal([n_classes]))

# Logits - h1W0 + b0
logits = tf.add(tf.matmul(h1, w0), b0)

# Define loss and optimizer
learning_rate = tf.placeholder(tf.float32)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
predict = tf.argmax(logits, 1)
correct_prediction = tf.equal(predict, tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

tf.summary.scalar('digit dnn', accuracy)

init = tf.global_variables_initializer()
merged = tf.summary.merge_all()

INFO:tensorflow:Summary name digit dnn is illegal; using digit_dnn instead.


In [12]:
def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))

In [13]:
import math
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    outout_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        outout_batches.append(batch)
        
    return outout_batches

In [14]:
#Define Hyperparameter
batch_size = 100
#epochs = 50
epochs = 10
learn_rate = 0.0002

In [15]:
train_batches = batches(batch_size, train_features, train_labels)


In [1]:
#Start Training Session
with tf.Session() as sess:
    sess.run(init)
    
    #start Tensorboard writer to log
    train_writer = tf.summary.FileWriter('./logg/', graph=tf.get_default_graph())
    test_writer = tf.summary.FileWriter('./logg/', graph=tf.get_default_graph())
    # Training cycle
    for epoch_i in range(epochs):

        # Loop over all batches
        for batch_features, batch_labels in train_batches:
            train_feed_dict = {
                features: batch_features,
                labels: batch_labels,
                learning_rate: learn_rate}
            sess.run(optimizer, feed_dict=train_feed_dict)
        
        summary = sess.run(merged,
                                      {features: batch_features, labels: batch_labels})
        train_writer.add_summary(summary, epoch_i)
        train_writer.flush()
        summary = sess.run(merged,
                                      {features: batch_features, labels: batch_labels})
        test_writer.add_summary(summary, epoch_i)
        test_writer.flush()
        print("tensorboard summary at #{}".format(epoch_i))

        # Print cost and validation accuracy of an epoch
#        accuracy = []
#        accuracy = (print_epoch_stats(epoch_i, sess, batch_features, batch_labels)).append()
        
        #summary, valid_accuracy = sess.run([merged, accuracy], feed_dict=feed_dict(False))
        #test_writter.add_summary(summary, epoch_i)
    
    predictions = sess.run(
                        predict, 
                        feed_dict={features: test_features})

NameError: name 'tf' is not defined

In [17]:
submissions = pd.DataFrame({"ImageId": list(range(1, len(predictions)+1)),
                             "Label": predictions})
submissions.to_csv("submission-1dnn-3-2048-50epoch-100batch.csv", index=False, header=True)

In [18]:
print(predictions)

[2 0 9 ... 3 9 2]


In [19]:
type(predictions)

numpy.ndarray